<a href="https://colab.research.google.com/github/aleks-haksly/Simulative/blob/main/Other/Analysis/RFM.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import pandas as pd
from sqlalchemy import text
from sqlalchemy import create_engine
import numpy as np
from google.colab import userdata
SQLALCHEMY_SILENCE_UBER_WARNING=1

In [2]:
# подключение к БД
engine = create_engine(userdata.get('simulative'))

In [3]:
# для удобства напишем функцию для загрузки результатов sql запросов
def select(sql):
    sql = text(sql)
    return pd.read_sql(sql, engine)

In [4]:
# запросим предагреггированные данные
sql = """
SELECT
  card as client_id,
  Max((SELECT max(datetime) :: date FROM bonuscheques) - datetime :: date) as days_passed,
  count(*) as cnt,
  sum(summ_with_disc) as summ
FROM
  bonuscheques
GROUP BY
  card
"""
data = select(sql)

In [5]:
# посмотрим, как выглядят данные
data.head()

,client_id,days_passed,cnt,summ
0,ece474c7-8a08-4853-8e6c-bc2d7043df7d,328,25,11577
1,2000200229544,234,2,1657
2,564a56ba-d8ae-43e4-ab1a-702198cb0c4c,317,14,12111
3,2000200171775,236,2,1033
4,2000200229094,165,2,1235


In [6]:
data.describe()

,days_passed,cnt,summ
count,9394.000000,9394.00000,9394.000000
mean,202.197147,4.09687,3416.820098
std,98.744462,6.43155,6388.046844
min,0.000000,1.00000,21.000000
25%,123.000000,1.00000,731.250000
50%,223.000000,2.00000,1586.000000
75%,290.000000,5.00000,3731.750000
max,332.000000,217.00000,162687.000000


Видим аномальный максимум в признаке cnt (Кто-то покупает почти каждый день?)

In [7]:
data["log_summ"] = data.summ.apply(np.log)

In [8]:
data.head()

,client_id,days_passed,cnt,summ,log_summ
0,ece474c7-8a08-4853-8e6c-bc2d7043df7d,328,25,11577,9.356776
1,2000200229544,234,2,1657,7.412764
2,564a56ba-d8ae-43e4-ab1a-702198cb0c4c,317,14,12111,9.401869
3,2000200171775,236,2,1033,6.940222
4,2000200229094,165,2,1235,7.118826


In [36]:
from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer, make_column_transformer
from sklearn.compose import make_column_selector as selector
from sklearn.impute import SimpleImputer
from sklearn.base import BaseEstimator, TransformerMixin
from sklearn.preprocessing import StandardScaler
from sklearn.cluster import DBSCAN
from sklearn.metrics import make_scorer
from sklearn.metrics import silhouette_score
from sklearn.model_selection import GridSearchCV

In [62]:
cols_to_test = ["days_passed", "cnt", "log_summ"]
cols_to_test_2 = ["days_passed", "cnt", "summ"]

In [86]:
col_selector = ColumnTransformer(
    transformers=[
        ('selector', "passthrough", cols_to_test)
    ],
    verbose_feature_names_out=False   # Оставляем оригинальные названия колонок
).set_output(transform="pandas")      # Трансформер будет возвращать pandas

In [87]:
col_transformer = ColumnTransformer(
    transformers=[
        ('StandardScaler', StandardScaler(), selector(dtype_include="number"))
    ],
    verbose_feature_names_out=False   # Оставляем оригинальные названия колонок
).set_output(transform='pandas')      # Трансформер будет возвращать pandas

In [88]:
class DBSCANWrapper(DBSCAN):
    def predict(self,X):
      return self.labels_.astype(int)

In [103]:
pipe_dt_basic = Pipeline(
    [
        ("col_selector", col_selector),
        ("col_transformer", col_transformer),
    ]
)

In [105]:
pipe_dt_dbscan = Pipeline(
    [
        ("col_selector", col_selector),
        ("col_transformer", col_transformer),
        ('simple_model', DBSCANWrapper(n_jobs=1))
    ]
)

In [106]:
dbscan_grid = {
    'simple_model__eps': [0.1, 0.3, 0.5, 0.7, 1, 1.5, 2, 2.5, 3, 4, 5],
    'simple_model__min_samples': [15, 20, 30, 35, 40, 45, 50],
    'simple_model__metric': ['euclidean', 'manhattan'],
    #'col_selector': [ColumnTransformer(transformers=[('selector', 'passthrough', ['days_passed', 'cnt', 'log_summ'])], verbose_feature_names_out=False), ColumnTransformer(transformers=[('selector', 'passthrough', ['days_passed', 'cnt', 'summ'])], verbose_feature_names_out=False)]
}

In [107]:
def score_func(df, labels, n_min=4):
    if len(set(labels)) >= n_min:
      return silhouette_score(df, labels)
    else:
      return -100

In [108]:
custom_metrics = make_scorer(score_func, greater_is_better=True)

In [109]:
search = GridSearchCV(pipe_dt_dbscan, dbscan_grid,
                      cv=[(slice(None), slice(None))], scoring=custom_metrics, )

In [110]:
search.fit(data, pipe_dt_basic.fit(data).transform(data))

GridSearchCV(cv=[(slice(None, None, None), slice(None, None, None))],
             estimator=Pipeline(steps=[('col_selector',
                                        ColumnTransformer(transformers=[('selector',
                                                                         'passthrough',
                                                                         ['days_passed',
                                                                          'cnt',
                                                                          'log_summ'])],
                                                          verbose_feature_names_out=False)),
                                       ('col_transformer',
                                        ColumnTransformer(transformers=[('StandardScaler',
                                                                         StandardScaler(),
                                                                         <sklearn.compose._column_transformer.make_column_selector object at 0x7916c3bb6050>)],
                                                          verbose_feature_names_out=False)),
                                       ('simple_model',
                                        DBSCANWrapper(n_jobs=1))]),
             param_grid={'simple_model__eps': [0.1, 0.3, 0.5, 0.7, 1, 1.5, 2,
                                               2.5, 3, 4, 5],
                         'simple_model__metric': ['euclidean', 'manhattan'],
                         'simple_model__min_samples': [15, 20, 30, 35, 40, 45,
                                                       50]},
             scoring=make_scorer(score_func))

In [111]:
print(f"Best parameter (CV score={search.best_score_:.5f}):")
print(search.best_params_)

Best parameter (CV score=-0.04177):
{'simple_model__eps': 0.1, 'simple_model__metric': 'euclidean', 'simple_model__min_samples': 35}


In [112]:
labels = search.predict(data)
unique, counts = np.unique(labels, return_counts=True)
print(np.asarray((unique, counts)).T)

[[  -1 9151]
 [   0   77]
 [   1  128]
 [   2   38]]
